In [1]:
import pandas as pd
import numpy as np
import torch

In [2]:
labels_df = pd.read_csv('data\labels.csv', header=None, names=['frame', 'exercise'])

In [3]:
# What exercises was performed? How was it perfomed?

In [4]:
# Dataset from where how it was collected and whats the datainside mean?

In [5]:
# where -001 mean no exercise was performed within those frames
# whole dataset it
labels_df

,frame,exercise
0,0,-1
1,1,-1
2,2,-1
3,3,-1
4,4,-1
...,...,...
8403,8403,-1
8404,8404,-1
8405,8405,-1
8406,8406,-1


In [6]:
landmarks_df = pd.read_csv('data\landmarks.csv')

In [7]:
# Dropping face keypoints as they are not necessary for movement classification

head_landmarks = [
    'NOSE', 'LEFT_EYE_INNER', 'LEFT_EYE', 'LEFT_EYE_OUTER',
    'RIGHT_EYE_INNER', 'RIGHT_EYE', 'RIGHT_EYE_OUTER',
    'LEFT_EAR', 'RIGHT_EAR', 'MOUTH_LEFT', 'MOUTH_RIGHT',
    # Adding leg landmarks
    'LEFT_KNEE', 'RIGHT_KNEE',
    'LEFT_ANKLE', 'RIGHT_ANKLE',
    'LEFT_HEEL', 'RIGHT_HEEL',
    'LEFT_FOOT_INDEX', 'RIGHT_FOOT_INDEX',
    # Adding finger landmarks
    'LEFT_PINKY', 'RIGHT_PINKY',
    'LEFT_INDEX', 'RIGHT_INDEX', 
    'LEFT_THUMB', 'RIGHT_THUMB'
]

landmarks_df = landmarks_df[~landmarks_df['landmark'].isin(head_landmarks)]

In [8]:
def process_landmarks_data(df):
    pivoted_df = df.pivot(index='frame_number', 
                         columns='landmark', 
                         values=['x', 'y', 'z'])
    
    pivoted_df.columns = [f'{landmark}_{coord}' 
                         for coord, landmark in pivoted_df.columns]
    
    return pivoted_df.reset_index()

In [9]:
processed_landmarks_df = process_landmarks_data(landmarks_df)

In [10]:
df = pd.merge(processed_landmarks_df, 
                    labels_df,
                    left_on='frame_number',
                    right_on='frame',
                    how='left')

if 'frame' in df.columns:
    df = df.drop('frame', axis=1)

In [11]:
df['exercise'] = pd.to_numeric(df['exercise'])

In [12]:
df.head(5)

,frame_number,LEFT_ELBOW_x,LEFT_HIP_x,LEFT_SHOULDER_x,LEFT_WRIST_x,RIGHT_ELBOW_x,RIGHT_HIP_x,RIGHT_SHOULDER_x,RIGHT_WRIST_x,LEFT_ELBOW_y,...,RIGHT_WRIST_y,LEFT_ELBOW_z,LEFT_HIP_z,LEFT_SHOULDER_z,LEFT_WRIST_z,RIGHT_ELBOW_z,RIGHT_HIP_z,RIGHT_SHOULDER_z,RIGHT_WRIST_z,exercise
0,0,0.494306,0.435416,0.461974,0.492193,0.320912,0.381529,0.367181,0.322582,0.507141,...,0.502203,-0.089719,0.000337,0.039939,-0.346921,-0.075649,-0.000382,0.042103,-0.336874,-1
1,1,0.492015,0.435435,0.462259,0.491906,0.320943,0.381469,0.366824,0.320433,0.521023,...,0.503574,-0.093571,0.000344,0.051836,-0.364749,-0.065798,-0.000387,0.052328,-0.318225,-1
2,2,0.492051,0.435449,0.462260,0.491421,0.320974,0.381461,0.366673,0.319015,0.517668,...,0.498068,-0.108150,0.000348,0.058875,-0.375957,-0.076405,-0.000389,0.058376,-0.319684,-1
3,3,0.491493,0.435497,0.462321,0.492260,0.320294,0.381467,0.366689,0.318692,0.525453,...,0.488508,-0.073606,0.001731,0.072559,-0.326744,-0.084063,-0.001726,0.055875,-0.333625,-1
4,4,0.491555,0.435549,0.462301,0.493636,0.318972,0.381528,0.366703,0.318285,0.523011,...,0.481967,-0.078110,0.002196,0.068983,-0.335180,-0.097579,-0.002175,0.054835,-0.344045,-1


In [13]:
# saving table for future dataloader 

df.to_csv('data\data.csv', index=False)

In [14]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8407 entries, 0 to 8406
Data columns (total 26 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   frame_number      8407 non-null   int64  
 1   LEFT_ELBOW_x      8407 non-null   float64
 2   LEFT_HIP_x        8407 non-null   float64
 3   LEFT_SHOULDER_x   8407 non-null   float64
 4   LEFT_WRIST_x      8407 non-null   float64
 5   RIGHT_ELBOW_x     8407 non-null   float64
 6   RIGHT_HIP_x       8407 non-null   float64
 7   RIGHT_SHOULDER_x  8407 non-null   float64
 8   RIGHT_WRIST_x     8407 non-null   float64
 9   LEFT_ELBOW_y      8407 non-null   float64
 10  LEFT_HIP_y        8407 non-null   float64
 11  LEFT_SHOULDER_y   8407 non-null   float64
 12  LEFT_WRIST_y      8407 non-null   float64
 13  RIGHT_ELBOW_y     8407 non-null   float64
 14  RIGHT_HIP_y       8407 non-null   float64
 15  RIGHT_SHOULDER_y  8407 non-null   float64
 16  RIGHT_WRIST_y     8407 non-null   float64


In [15]:
df = df.drop(columns='frame_number')
df.to_csv('data\data_for_model.csv', index=False)